## Imports

In [62]:
import os
import nibabel as nib
import glob

### Reading all BraTS21 examples, change the root_path to you base address

In [63]:
root_path = r'C:\Users\DELL\Graduation Project\Datasets\Brats21'                     #Change this to dataset base address
data_list = sorted(glob.glob(root_path + '/*'))                                      #list of paths of the inside files

#### The following function is responsible for returning the indices of the brain of the volume that contains foreground voxels.

In [64]:
def find_brain_width_wise(dep, hei, i, img):        #cropping width wise
    slice2D = img.get_fdata()[:, i, :]
    for j in range(hei):
        for k in range(dep):
            if slice2D[j, k] != 0:
                return i
    return 0

def find_brain_height_wise(dep, wid, i, img):      #cropping height wise
    slice2D = img.get_fdata()[i, :, :]
    for j in range(wid):
        for k in range(dep):
            if slice2D[j, k] != 0:
                return i
    return 0

def find_brain_depth_wise(wid, hei, i, img):        #cropping depth wise
    slice2D = img.get_fdata()[:, :, i]
    for j in range(wid):
        for k in range(hei):
            if slice2D[j, k] != 0:
                return i
    return 0

#### Creating results folder that will contain the cropped volumes

In [65]:
# Create results folder
results_path = r'C:\Users\DELL\Graduation Project\Datasets\xx'
if not os.path.exists(results_path):
    os.makedirs(results_path)

In [66]:
sorted(glob.glob(data_list[0] + '/*'))

['C:\\Users\\DELL\\Graduation Project\\Datasets\\Brats21\\BraTS2021_00000\\BraTS2021_00000_flair.nii.gz',
 'C:\\Users\\DELL\\Graduation Project\\Datasets\\Brats21\\BraTS2021_00000\\BraTS2021_00000_seg.nii.gz',
 'C:\\Users\\DELL\\Graduation Project\\Datasets\\Brats21\\BraTS2021_00000\\BraTS2021_00000_t1.nii.gz',
 'C:\\Users\\DELL\\Graduation Project\\Datasets\\Brats21\\BraTS2021_00000\\BraTS2021_00000_t1ce.nii.gz',
 'C:\\Users\\DELL\\Graduation Project\\Datasets\\Brats21\\BraTS2021_00000\\BraTS2021_00000_t2.nii.gz']

In [74]:
depth_cropper_type = input("In case of the depth cropping\n"
                           "Do you want it to be cropped based on the brain voxels or the tumor voxels?\n"
                           "please enter either 'brain', or 'tumor' \n")

In case of the depth cropping
Do you want it to be cropped based on the brain voxels or the tumor voxels?
please enter either 'brain', or 'tumor' 
tumor


### Looping in every example and in every module in the volume to crop it to contain only the brain part without the background slices. In addition, we will also crop it depthwise to contain the tumorous slices only

In [71]:
#Iterating for every example in the folder
for example in range(0, 1):                           # The number of example I want to crop and save from the data
    modules_list = sorted(glob.glob(data_list[example] + '/*'))          #modules list of a single volume
    vol_path = modules_list[0]                        #It doesn't matter which mode, just no seg (tumor), as we want the brain
    img = nib.load(vol_path)                          #Flair volume (it doesn't matter)
    
    seg_vol_path = modules_list[1]                    #Segmentation volume (second volume)
    img_tumor = nib.load(seg_vol_path)

    height, width, depth = img.shape
    
    filled_slices_width = []                              #slices indices with foreground values
    filled_slices_height = []
    filled_slices_depth = []

    #Iterating in the dimension of interest, which we will extract foreground slices from, repeating it in the 3 dimension (3D Volume)

    if (depth_cropper_type == 'brain'):
        depth_img = img
    elif (depth_cropper_type == 'tumor'):
        depth_img = img_tumor
    else:
        print("No cropping has been made.")
        break
    
    for i in range(width):
        width_idx = find_brain_width_wise(depth, height, i, img)       #img not img_tumor in the width and height
        if width_idx != 0:
            filled_slices_width.append(width_idx)

    for i in range(height):
        height_idx = find_brain_height_wise(depth, width, i, img)
        if height_idx != 0:
            filled_slices_height.append(height_idx)
        

    for i in range(depth):           
        depth_idx = find_brain_depth_wise(width, height, i, depth_img)     #depth_img is either img or img_tumor
        if depth_idx != 0:
            filled_slices_depth.append(depth_idx)

    min_wid_idx, max_wid_idx = filled_slices_width[0], filled_slices_width[-1]
    min_hei_idx, max_hei_idx = filled_slices_height[0], filled_slices_height[-1]
    min_dep_idx, max_dep_idx = filled_slices_depth[0], filled_slices_depth[-1]

    #Cropping step, iterating for every module in the same example to be cropped.
    for module_path in modules_list:
        module_vol = nib.load(module_path)
        cropped_vol = module_vol.get_fdata()[min_hei_idx : (max_hei_idx+1),
                                             min_wid_idx : (max_wid_idx+1),
                                             min_dep_idx : (max_dep_idx+1)]

        nifti_img =  nib.Nifti1Image(cropped_vol, module_vol.affine)            # to save this 3D (ndarry) numpy

        #to make the naming of the resulted folders and files the same as the original naming
        vol_new_path = results_path + '/' + data_list[example].split('\\')[-1]
        if not os.path.exists(vol_new_path):
            os.makedirs(vol_new_path)

        module_new_path = vol_new_path + '/' + module_path.split('\\')[-1]
        nib.save(nifti_img, module_new_path)

### Testing random module

##### Non-cropped volume

In [21]:
test_path = r'C:\Users\DELL\Graduation Project\Slices\Brats21\BraTS2021_00000\BraTS2021_00000_t1.nii.gz'
img_test = nib.load(test_path)

img_test.shape

(240, 240, 155)

##### brain cropped volume

In [22]:
test_path = r'C:\Users\DELL\Graduation Project\0-100 Niftis Cropped-3D-T Brats\BraTS2021_00000\BraTS2021_00000_t1.nii.gz'
img_test = nib.load(test_path)

img_test.shape

(136, 171, 146)

##### tumor cropped volume

In [20]:
test_path = r'C:\Users\DELL\Graduation Project\Datasets\0-100 Imgs\0-100 Flair_imgs\Flair_imgs_middle_only\BraTS2021_00000\BraTS2021_00000_t1.nii.gz'
img_test = nib.load(test_path)

img_test.shape

(136, 171, 47)

#### Image shape was (240, 240, 155) Before cropping, now it's (136, 171, 146), and (136, 171, 47) for this example (Random)